# Preprocessing of stock data

* Fetch raw stock data from influxdb, 
* Create technical indicators
   * moving average 24 hr
   * moving average 7 days
   * moving average 90 days
   * investigate further which KPIs are good? e.g. the Bollinger Bands 
   * Trend indicators for directions are volume based. E.g. Accumulation Distribution Line, Chaikin Money Flow, the Money Flow Index (MFI) or On Balance Volume (OBV).
   * https://school.stockcharts.com/doku.php?id=trading_strategies:bollinger_band_squeeze

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from datetime import datetime
import pandas as pd
import numpy as np
from scipy import stats

import plotly.plotly as plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import utils.stockler_influx as db
import utils.charting as chart_helper

### Fetching stock data from influx
We fetch multiple stocks at once. 
We'll take the candle price data and volume. There are two invervals one is custom and the other is the daily data. 

In [ ]:
symbols = ['TSLA', 'XBIO']
pattern = '|'.join(symbols)
options = {
    'symbol_regex':f'/{pattern}/', 
    'min_date': datetime(2018,2,1)
}

In [ ]:
columns = [
    """ MIN("low") as "low" """,
    """ MAX("high") as "high" """,
    """ FIRST("average_price") as "open" """,
    """ LAST("average_price") as "close" """,
    """ SUM("volume") as "volume" """,
]
raw_1d_data_by_stock = db.select_from_stocks(',\n'.join(columns), epoch='1d', **options)
raw_data_by_stock = db.select_from_stocks(',\n'.join(columns), epoch='1m', **options)

### Plotly    - (Boiler plate settings)

In [ ]:
init_notebook_mode(connected=False)

### Add some indicators

I'll add a few technical indicators such as the bollinger bands and the simple moving averages.
We'll then scale all values to make them as close to 0 as possible.

Pricing data's meaning is the % change in stock movement.
Bollinger bands are meaningfull to see the risky zone around the simple moving price. Therefore it's 
the percenatge distance between the moving average and the upper/lower bands.

volume: I think the volume compared to the mean daily volume could be intresting. 

In [ ]:
indicator_data_by_stock = {}

for symbol, data in raw_data_by_stock.items():
    data_1d = raw_1d_data_by_stock[symbol]

    
    bb_df = chart_helper.calculate_bollinger_bands('close', data, 10)
    bb_df = bb_df.rename(columns={'avg': 'bb_avg', 'lower': 'bb_low', 'upper': 'bb_high'})

    moving_avg_7 = chart_helper.calculate_moving_average('close', data_1d, 7)
    moving_avg_7 = moving_avg_7.rename(columns={'average': 'avg_7d'})
    
    moving_avg_90 = chart_helper.calculate_moving_average('close', data_1d, 90)
    moving_avg_90 = moving_avg_90.rename(columns={'average': 'avg_90d'})
    moving_avg = pd.concat([moving_avg_7, moving_avg_90], axis=1).dropna()
    
    normalized_data = pd.concat([data])
    normalized_data = pd.concat([normalized_data, bb_df], axis=1).dropna()
    normalized_data = normalized_data.join(moving_avg, how='outer').bfill().drop(moving_avg.index).dropna()
    normalized_data['volume'] = data['volume']
    
    #removes outliers
    #normalized_data = normalized_data[(np.abs(stats.zscore(normalized_data)) < 3).all(axis=1)]
    
    print(normalized_data.max())
    
    indicator_data_by_stock[symbol] = normalized_data
print(indicator_data_by_stock.keys())

In [ ]:
for symbol, data in indicator_data_by_stock.items():
    file_name = f'{symbol}_{data.index.min().date().isoformat()}_{data.index.max().date().isoformat()}'
    file_path = f'~/data/{file_name}.csv'
    data.to_csv(file_path, index_label='datetime')
    print('exported ' + file_path)


### Time to do some plotting

In [ ]:
figure = chart_helper.get_figure_with_sub_chart(has_rangeselector=False, y_axis_range=None)

for symbol, stock in indicator_data_by_stock.items():

    chart_helper.add_candle_chart(
        figure, 
        open=stock.open, 
        high=stock.high, 
        low=stock.low, 
        close=stock.close, 
        x_data=stock.index,
        name=symbol
    )

    chart_helper.add_bar_chart(
        figure, 
        x_data=stock.index, 
        y_data=stock.volume, 
        yaxis='y2', 
        name=f'Volume_{symbol}'
    )

    chart_helper.add_scatter_chart(
        figure, y_data=stock.bb_high,
        x_data=stock.index, name='BB_'+symbol
    )
    chart_helper.add_scatter_chart(
        figure, y_data=stock.bb_low, x_data=stock.index,
        name=f'BB_{symbol}', showlegend=False
    )
    chart_helper.add_scatter_chart(
        figure, y_data=stock.bb_avg, x_data=stock.index,
        name=f'BB_avg_{symbol}', color='#000'
    )
    chart_helper.add_scatter_chart(
        figure, y_data=stock.avg_7d,
        x_data=stock.index, name=f'avg_7_{symbol}', color='#3492B7'
    )

    chart_helper.add_scatter_chart(
        figure, y_data=stock.avg_90d,
        x_data=stock.index, name=f'avg_90_{symbol}', color='#3492B7'
    )

iplot(figure, filename = 'candlestick-test-3', validate = False)